In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
plt.style.use('ggplot') 
import calendar
import quandl
import pickle
import math
from datetime import datetime, timedelta
import re  
import psycopg2
import sys
import matplotlib.gridspec as gridspec
import sqlalchemy
from sqlalchemy.sql import select, and_, or_, not_, desc, asc
from sqlalchemy import Table, Column, Integer, DateTime, String,Float, ForeignKey
from sqlalchemy.orm import sessionmaker
from sklearn import preprocessing

In [3]:
# fig_size = plt.rcParams["figure.figsize"]
# fig_size[0] = 10
# fig_size[1] = 10
# plt.rcParams["figure.figsize"] = fig_size

In [4]:
def connect(user, password, db, host='localhost', port=5432):
    '''Returns a connection and a metadata object'''
    # We connect with the help of the PostgreSQL URL
    url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)

    # The return value of create_engine() is our connection object
    con = sqlalchemy.create_engine(url, client_encoding='utf8')

    # We then bind the connection to MetaData()
    meta = sqlalchemy.MetaData(bind=con)

    return con, meta

CREATE TABLES 

In [5]:
con, meta = connect('postgres', '', 'robotdb')

mkt_trend = Table('Market_trend', meta,
                  Column('coin', String, primary_key=True),
                  Column('date', DateTime, primary_key=True),
                  Column('screen', Integer, primary_key=True),
                  Column('dif_current', Float),
                  Column('dif_base', Float),
                  Column('d_dif', Float),
                  Column('theta_current', Float),
                  Column('theta_base', Float),
                  Column('d_theta', Float),
                  Column('long_dif', Float),
                  Column('max_growth', Float),
                  Column('max_loss', Float),
                  Column('max_price', Float),
                  Column('min_price', Float),
                  Column('max_rel', Float),
                  Column('min_rel', Float),
                  Column('log_ret', Float),
                  Column('log_ret_p', Float),
                  Column('histogram', Float),
                  Column('ema_dif', Float),
                  Column('rsi', Float),
                  Column('dif_sma', Float),
                  Column('max_growth_p', Float),
                  Column('obv', Float),
                  Column('vote', Integer)
                  )

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
yearsFmt = mdates.DateFormatter('%Y')

In [6]:
def get_mkt_trend(coin=None, date='2019-12-31', screen=1):
    if coin:
        s = select([mkt_trend]).\
            where(and_(mkt_trend.c.coin == coin, mkt_trend.c.date <= date, mkt_trend.c.screen == screen)).\
            order_by(desc(mkt_trend.c.date))
    else:
        s = select([mkt_trend]).\
            where(and_(mkt_trend.c.date <= date, mkt_trend.c.screen == screen)).\
            order_by(desc(mkt_trend.c.date))
    rows = con.execute(s)
    mkt_trend_df = pd.DataFrame(rows.fetchall()).iloc[::-1]
    mkt_trend_df.columns = rows.keys()
    return mkt_trend_df
get_mkt_trend()

,coin,date,screen,dif_current,dif_base,d_dif,theta_current,theta_base,d_theta,long_dif,...,min_rel,log_ret,log_ret_p,histogram,ema_dif,rsi,dif_sma,max_growth_p,obv,vote
1889,USDT_BTC,2017-05-14 14:00:00,1,-2.796446,-1.746831,0.600868,-1.090036,-1.049690,0.038436,117.324542,...,-0.024458,-0.015680,-0.028238,9.153076,-0.004250,59.154887,17.916067,NaN,0.000000e+00,1
1888,USDT_BTC,2017-05-14 18:00:00,1,0.380872,-2.796446,-1.136198,-1.097405,-1.056191,0.039021,117.324542,...,-0.022260,0.012102,-0.003578,6.353208,0.009530,56.172443,20.724558,NaN,1.962037e+06,0
1887,USDT_BTC,2017-05-14 22:00:00,1,5.309024,0.380872,12.939142,-1.105253,-1.064014,0.038757,117.324542,...,-0.029389,0.009330,0.021433,8.039552,0.012428,51.971609,20.035232,NaN,1.780218e+06,0
1886,USDT_BTC,2017-05-15 02:00:00,1,5.769352,5.309024,0.086707,-1.113599,-1.074446,0.036441,117.324542,...,-0.001131,-0.019845,-0.010514,10.856140,-0.021066,58.117693,24.980232,NaN,-1.069966e+06,0
1885,USDT_BTC,2017-05-15 06:00:00,1,0.886452,5.769352,-0.846352,-1.120941,-1.085765,0.032397,117.324542,...,-0.008033,-0.027417,-0.047262,8.419232,-0.028006,54.995666,17.639511,NaN,-1.305461e+06,0
1884,USDT_BTC,2017-05-15 10:00:00,1,-1.264667,0.886452,-2.426662,-1.125067,-1.093601,0.028773,117.324542,...,-0.016858,0.009115,-0.018301,1.022328,-0.016085,51.170164,8.197044,NaN,1.000982e+06,0
1883,USDT_BTC,2017-05-15 14:00:00,1,-3.440598,-1.264667,1.720557,-1.126521,-1.099038,0.025006,107.223513,...,-0.016521,-0.009688,-0.000573,-1.901981,-0.016290,50.276463,3.252228,NaN,-5.878209e+06,0
1882,USDT_BTC,2017-05-15 18:00:00,1,-6.104384,-3.440598,0.774222,-1.124799,-1.100855,0.021750,107.223513,...,-0.010984,0.001801,-0.007887,-4.042176,-0.017905,48.069405,-6.131850,NaN,3.479247e+06,0
1881,USDT_BTC,2017-05-15 22:00:00,1,-6.886715,-6.104384,0.128159,-1.120065,-1.098762,0.019389,107.223513,...,-0.018649,0.009243,0.011044,-6.037576,-0.008164,44.952684,-18.962017,NaN,-2.219618e+06,0
1880,USDT_BTC,2017-05-16 02:00:00,1,-9.478976,-6.886715,0.376415,-1.113428,-1.094556,0.017242,107.223513,...,-0.006299,-0.011454,-0.002211,-5.526262,-0.018320,40.639619,-22.452963,NaN,-2.106590e+06,0


In [ ]:
def get_strength_index():
    tickers_df_two = get_tickers()
    for c in tickers_df_two.coin.unique():
        tickers_df_two_c = tickers_df_two[tickers_df_two['coin'] == c]
        print(c)
        # 6 * 4h = 24h
        delta_t = 1
        si_np = [np.nan]
        for index in range(len(tickers_df_two_c)):
            base_price = tickers_df_two_c.iloc[index].price
            if index >= (len(tickers_df_two_c)-delta_t):
                break
            current_price = tickers_df_two_c.iloc[index+1].price
            volume = tickers_df_two_c.iloc[index+1].volume
            si = (current_price - base_price) * volume
            si_np.append(si)
        tickers_df_two_c['strength'] = si_np
    return tickers_df_two_c.dropna().drop(['price', 'volume', 'screen'], axis=1)

FETCH DATA FROM DATABASE AND JOIN TABLES 
MKTTREND - PRICE - MACDS - RSI - SMA

In [ ]:
def manipulate_mkt_data():
    #Market Trend
    mkt_trend_df = get_mkt_trend()
    #Prices
    tickers_df_two = get_tickers()
    #MACD
    macds_df = get_macds()
    
    #Merge prices and macds
    tickers_df = pd.merge(tickers_df_two, macds_df, how='inner', left_on=['date', 'coin', 'screen'],
                          right_on=['date', 'coin', 'screen'])
    #Calculate relationship between Price and EMA
    tickers_df['ema_dif'] = np.log(tickers_df['price']/tickers_df['ema12'])
    #Merge Prices and MKT TREND
    df = pd.merge(mkt_trend_df, tickers_df, how='inner', left_on=['date', 'coin', 'screen'],
                  right_on=['date', 'coin', 'screen'])
    #RSI
    rsis_df = get_rsis()
    df = pd.merge(df, rsis_df, how='inner', left_on=['date', 'coin', 'screen'],
                  right_on=['date', 'coin', 'screen'])
    #SMA
    sma_df = get_smas()
    df = pd.merge(df, sma_df, how='inner', left_on=['date', 'coin', 'screen'],
                  right_on=['date', 'coin', 'screen'])
    full_data_set = df.drop(['sma20_theta', 'screen'], axis=1)
    dates_ = full_data_set[['date']]
    shifted_parms = full_data_set[['coin', 'dif_current', 'theta_current']].shift(2)
    shifted_parms['date'] = dates_
    full_data_set = pd.merge(full_data_set, shifted_parms, how='inner', left_on=['date', 'coin'],
                             right_on=['date', 'coin'])
    full_data_set['dif_current'] = full_data_set['dif_current_x']
    full_data_set['theta_current'] = full_data_set['theta_current_x']
    full_data_set['dif_base_p'] = full_data_set['dif_current_y']
    full_data_set['theta_base_p'] = full_data_set['theta_current_y']
    full_data_set = full_data_set.drop(['dif_current_x', 'dif_current_y',
                                        'theta_current_x', 'theta_current_y'], axis=1)
    full_data_set['dif_sma'] = full_data_set['sma5'] - full_data_set['sma20']
    df_model = pd.merge(full_data_set, get_strength_index(), how='inner',
                    left_on=['date', 'coin'], right_on=['date', 'coin'])
    df_model = df_model.sort_values(['date'])
    df_model['theta_ema'] = df_model.theta_current.ewm(span=6,min_periods=6,adjust=True,ignore_na=False).mean()
    df_model['dif_ema'] = df_model.dif_current.ewm(span=6,min_periods=6,adjust=True,ignore_na=False).mean()
    df_model['strength_ema'] = df_model.strength.ewm(span=6,min_periods=6,adjust=True,ignore_na=False).mean()
    df_model = df_model.dropna()
    df_model.loc[df_model['max_growth'] >= 0.1, 'growth_sign'] = 1
    df_model.loc[df_model['max_growth'] < 0.1, 'growth_sign'] = 0
    df_model.loc[df_model['max_loss'] <= -0.05, 'loss_sign'] = 1
    df_model.loc[df_model['max_loss'] > -0.05, 'loss_sign'] = 0
    dates_ = df_model[['date']]
    shifted_parms = df_model[['coin', 'max_growth', 'max_loss']].shift(1)
    shifted_parms['date'] = dates_
    df_model = pd.merge(df_model, shifted_parms, how='inner', left_on=['date', 'coin'], right_on=['date', 'coin'])
    df_model['max_growth'] = df_model['max_growth_x']
    df_model['max_loss'] = df_model['max_loss_x']
    df_model['max_growth_p'] = df_model['max_growth_y']
    df_model['max_loss_p'] = df_model['max_loss_y']
    df_model = df_model.drop(['max_growth_x', 'max_growth_y', 'max_loss_x', 'max_loss_y'], axis=1)
    df_model['log_ret'] = np.log(df_model.price) - np.log(df_model.price.shift(1))
    df_model['log_ret_p'] = np.log(df_model.price) - np.log(df_model.price.shift(2))
    df_model = df_model[~(np.abs(df_model.d_dif-df.d_dif.mean())>(3*df_model.d_dif.std()))]
    df_model = df_model[~(np.abs(df_model.strength_ema-df_model.strength_ema.mean())>(3*df_model.strength_ema.std()))]
    df_model = df_model[~(np.abs(df_model.d_theta-df.d_theta.mean())>(3*df_model.d_theta.std()))]
    df_model = df_model.dropna()
    df_model = df_model.sort_values(['date'])
    df_model = df_model.drop(['coin', 'date', 'vote'], axis=1)
    p_price = 0
    df_model['obv'] = calculate_obv(df_model)
    df_model['max_rel'] = np.log(df_model['max_price']/ df_model['price'])
    df_model['min_rel'] = np.log(df_model['min_price']/ df_model['price'])
    df_model['rsi'] = df_model['rsi']/100
    df = df_model.drop(['macd_line', 'signal_line', 'dif_ema', 'loss_sign',
                           'theta_ema', 'max_loss', 'max_growth',
                         'd_theta', 'max_loss_p', 'd_dif', 'max_price', 'min_price',
                         'price', 'sma5', 'sma20', 'ema12', 'ema_26', 'sma5_theta',
                         'volume', 'strength', 'macd_line', 'signal_line', 'dif_base_p',
                   'theta_base_p'] ,axis=1)
    return df
    
df = manipulate_mkt_data().reset_index().drop(['index'], axis=1)
df.head()

In [ ]:
strength_ema, obv

In [ ]:
train = df.iloc[0:math.floor(len(df)*0.75)].copy()
test = df.iloc[math.floor(len(df)*0.75):len(df)].copy()

scale_columns = ['theta_current', 'dif_current', 'theta_base','dif_base',
                 'dif_sma', 'strength_ema', 'obv']
    

scaler = preprocessing.Normalizer().fit(train[scale_columns])
#STANDARD SCALER
# scaler = preprocessing.StandardScaler().fit(train[scale_columns])
train[scale_columns] = scaler.transform(train[scale_columns])
test[scale_columns] = scaler.transform(test[scale_columns])
#NORMALIZATION
# train[scale_columns] = preprocessing.normalize(train[scale_columns])
# test[scale_columns] = preprocessing.normalize(test[scale_columns])


X_train, y_train = train.drop(['growth_sign'], axis=1), train.growth_sign.values
X_test, y_test =  test.drop(['growth_sign'], axis=1), test.growth_sign.values
X_train.head()

In [ ]:
plt.figure(figsize=(12,8*4))
gs = gridspec.GridSpec(7, 4)
for i, cn in enumerate(train.drop(['growth_sign'], axis=1).columns):
    ax = plt.subplot(gs[i])
    sns.distplot(train[cn][train.growth_sign == 1], bins=30, color='blue')
    sns.distplot(train[cn][train.growth_sign == 0], bins=30, color='red')
    ax.set_xlabel('')
    ax.set_title('feature: ' + str(cn))
plt.show()

In [ ]:
# #EXAMPLE FOR SCALE NEW ENTRY
t_transform = scaler.transform(df.loc[0,scale_columns].values.reshape(1,-1))
print(t_transform)

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
oversampler = SMOTE(random_state=42)
X_train_os, Y_train_os = oversampler.fit_sample(X_train,y_train)

# RANDOM FOREST
print('Random Forest')
param_grid = {'max_depth': np.arange(1, 20), 'min_samples_leaf':np.arange(1, 8),}
model = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='precision')
model.fit(X_train_os, Y_train_os)
Y_predict = model.predict(X_test)

print(model.best_params_)
cm = confusion_matrix(y_test,Y_predict)
print('Precision: %s' % float(cm[1][1]/(cm[1][1]+cm[0][1])))
print(cm)

print('---------------------------------------------------')

print('AdaBoost')
# ADABOOST
param_grid = {'n_estimators': np.arange(1, 32)}
model = GridSearchCV(AdaBoostClassifier(random_state=42), param_grid, cv=5, scoring='precision')
model.fit(X_train_os, Y_train_os)
Y_predict = model.predict(X_test)

print(model.best_params_)

cm = confusion_matrix(y_test,Y_predict)
print('Precision: %s' % float(cm[1][1]/(cm[1][1]+cm[0][1])))
print(cm)

In [ ]:
oversampler = SMOTE(random_state=42)
X_train_os, Y_train_os = oversampler.fit_sample(X_train,y_train)

s_model = AdaBoostClassifier(random_state=42, n_estimators=30)
s_model.fit(X_train_os, Y_train_os)
Y_predict = s_model.predict(X_test)
cm = confusion_matrix(y_test,Y_predict)
print('Precision: %s' % float(cm[1][1]/(cm[1][1]+cm[0][1])))
print(cm)
FP, TP, thresholds = roc_curve(y_test,Y_predict)
roc_auc = auc(FP, TP)
print (roc_auc)